In [ ]:
import altair as alt
import pandas as pd
import numpy as np
import geopandas as gpd
import json

In [ ]:
emisPop = pd.read_pickle('emisPop.pkl')
emisPop = emisPop.loc[emisPop.year=='2014']

In [ ]:
brush = alt.selection(type='interval')

pts = alt.selection(type="single", encodings=['x'])

# the top scatterplot
points = alt.Chart().mark_point().encode(
    x=alt.X('popGrowth_5yr:Q', axis=alt.Axis(title='5-year Average Population Growth Rate (%)')),
    y=alt.Y('co2perCap:Q', axis=alt.Axis(title='CO2 Emissions (metric tons per capita)')),
    color=alt.condition(brush,
                        alt.Color(
                            'continent:N',
                            legend=alt.Legend(
                                title='Continent',
                            )
                        ),
                        alt.value('lightgray'))
).properties(
    selection=brush,
    width=800
).transform_filter(
    pts.ref()
)

# the bottom bar plot
bars = alt.Chart().mark_bar().encode(
    y=alt.Y('continent:N', axis=alt.Axis(title='Continent')),
    color='continent:N',
    x=alt.X('count(continent):Q', axis=alt.Axis(title='Number of Countries'))
).transform_filter(
    brush.ref() # the filter transform uses the selection
                # to filter the input data to this chart
).properties(
    selection=pts,
    width=550,
    height=200
)

chart = alt.vconcat(points, bars, data=emisPop)
chart.save('co2_chart.html')
chart

In [ ]:
json_state = (emisPop.rename(columns={'country':'name'})
     .groupby(['continent'], as_index=True)
     .apply(lambda x: x[['name', 'co2','co2perCap']].to_dict('r'))
     .reset_index()
     .rename(columns={0:'children', 'continent':'name'})
     .to_json(orient='records'))

with io.open('co2.json', 'w', encoding='utf8') as outfile:
    str_out = json.dumps(json.loads(json_state), indent=2, sort_keys=True)
    outfile.write('{"children":')
    outfile.write(str_out)
    outfile.write('}')